In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


### Data Reading

In [0]:
df = spark.read.format("parquet").load("s3://argha-associatede-s3/bronze/customers/")

display(df)

In [0]:
df1 = df.withColumn("domains",split(col("email"), "@")[1])

display(df1)

In [0]:
df_groupby = df1.groupBy("domains")\
    .agg(count("customer_id")\
        .alias("num_cust"))\
            .orderBy("num_cust",ascending=False)

df_groupby.display()

In [0]:
df_gmail = df1.filter(col("domains") == "gmail.com")

display(df_gmail)

In [0]:
df_yahoo = df1.filter(col("domains") == "yahoo.com")

display(df_yahoo)

In [0]:
df_hotmail = df1.filter(col("domains") == "hotmail.com")

display(df_hotmail)

In [0]:
df1 = df1.withColumn("full_name",concat("first_name",lit(" "), "last_name"))
df1 = df1.drop("first_name", "last_name")

df1.display()


### Data Writing

In [0]:
df1.write.format("delta").mode("overwrite").save("s3://argha-associatede-s3/silver/customers/")